# // Getting NFL Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to rely on the `requests` and `BeautifulSoup` libraries to glean information from:

- [FantasyPros](https://www.fantasypros.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)
- [FFToday](http://www.fftoday.com/stats/)
- [The Football Database](https://www.footballdb.com/fantasy-football/index.html)

**Editor's Note:** After researching several different options for quite some time and attempting to use many of them, I will be going with **FantasyPros** for reasons outlined below.

In [104]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
%matplotlib inline

## FantasyPros
I'm choosing to go with this FantasyPros for the following reasons:
- You don't have to register to access the data
- Separately, I am currently a user and like their site overall
- Has available .5 PPR scoring data (FanDuel does .5 PPR so DFS appeal, and it will be most applicable for most leagues)
- The Football Database seemed like it was missing quite a few players from each week
- FFToday scoring was inconsistent in many places for .5 PPR scoring

#### Following the FanDuel vein, since Daily Fantasy Sports (DFS) is where the biggest value is going to be coming from, I will be scraping data for the following positions and referring to them from here on out by the name in parentheses:
1. Quarterback (QB)
2. Running Back (RB)
3. Wide Receiver (WR)
4. Tight End (TE)
5. Defense / Special Teams (DST)


> - For a more detailed breakdown on positional scoring, please reference FanDuel's scoring and rules reference [here](https://www.fanduel.com/rules).
- For a quick reference on what each column means, check out ESPN's stat reference [here](http://www.espn.com/nfl/news/story?id=2128923).

___
### Scraping for our QBs
Going to walk through the specific steps for this first position then blast through the others.

1. Let's make some empty lists to throw all of our data into:

In [105]:
player = []
pass_comp = []
pass_att = []
pass_pct = []
pass_yds = []
yds_per_att = []
pass_TD = []
pass_INT = []
sacks_taken = []
rush_att = []
rush_yds = []
rush_TD = []
fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

In [106]:
# Make sure to incoporate this into the scrape!
week = []
year = []

2. And a list of those lists to make `dataframe` creation nice and easy:

In [107]:
qb_stats_lists = [player, pass_comp, pass_att, pass_pct, pass_yds, 
                  yds_per_att, pass_TD, pass_INT, sacks_taken, 
                  rush_att, rush_yds, rush_TD, fumbles_lost, 
                  active, fpoints, fpoints_g, own_pct]



3. Doing the actual scrape by going through each week of the season (via each URL iteration), grabbing the right `table`, iterating through each `row` and then `cell`, and putting that data into the appropriate `lists`.

In [108]:
for week_number in range(1,6):

    res = requests.get('https://www.fantasypros.com/nfl/stats/qb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')

    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(qb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    # And adding a 1 second delay 
    sleep(1)

4. Let's make an empty `dataframe` and then fill it using our list of lists:

In [109]:
qbdf = pd.DataFrame(columns= ['player', 'pass_comp', 'pass_att', 'pass_pct',
                              'pass_yds', 'yds_per_att', 'pass_TD', 'pass_INT', 
                              'sacks_taken', 'rush_att', 'rush_yds', 'rush_TD', 
                              'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
qbdf

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct


In [110]:
for index, column in enumerate(qbdf.columns):
    qbdf[column] = qb_stats_lists[index]

5. Let's add on our `week` and `year` information as well as a separate `team` column from our `player` column (to later be used to bring in opponent data).

In [111]:
# Week
qbdf['week'] = week
qbdf['week'] = qbdf.week.astype(str)

In [112]:
team = []
for individual in qbdf['player']:
    team.append(re.findall('\(([^\)]+)\)', individual)[0])
qbdf['team'] = team

6. And let's just clean up that `player` name a bit too with another `regex`:

In [113]:
qbdf['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', quarterback) for quarterback in qbdf['player']]]

7. While we're at the cleanup, we're going to need to do some with the `own_pct`:

In [114]:
qbdf['own_pct'] = qbdf.own_pct.apply(lambda x: x.strip('%'))
qbdf.own_pct.replace(to_replace='', value='0.0', inplace=True)

8. Since most if not all of these values were actually read in as `strings`, we'll need to convert them to numeric columns (with the exceptions of `player`, `team`, and `week`.

In [115]:
for x in [col for col in qbdf.columns if col not in ['player','week', 'team', 'year']]:
    qbdf[x] = qbdf[x].astype(float)

**Since we're going to likely be wanting to clean up our other dataframes in the same way, let's make a function to clean up our future code:**

In [116]:
def table_cleaner(df):
    # Adding 'week' as a string
    df['week'] = week
    df['week'] = df.week.astype(str)
    
    # Cleaning 'team'
    team = []
    for individual in df['player']:
        team.append(re.findall('\(([^\)]+)\)', individual)[0])
    df['team'] = team
    
    # Removing team from 'player'
    df['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', person) for person in df['player']]]    
    
    # Converting 'own_pct' to float
    df['own_pct'] = df.own_pct.apply(lambda x: x.strip('%'))
    df.own_pct.replace(to_replace='', value='0.0', inplace=True)    
    
    # Converting everything else to float where appropriate
    for x in [col for col in df.columns if col not in ['player','week', 'team', 'year']]:
        df[x] = df[x].astype(float)
    
    return df

In [117]:
# Checking to see if we'll be able to manipulate it how we want
qbdf.sort_values('fpoints', ascending=False).head(10)

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
460,Mitch Trubisky,19.0,26.0,73.1,354.0,13.6,6.0,0.0,1.0,3.0,53.0,0.0,0.0,1.0,43.5,43.5,25.3,4,CHI
30,Ryan Fitzpatrick,21.0,28.0,75.0,417.0,14.9,4.0,0.0,0.0,12.0,36.0,1.0,0.0,1.0,42.3,42.3,28.1,1,TB
262,Drew Brees,39.0,49.0,79.6,396.0,8.1,3.0,0.0,1.0,3.0,7.0,2.0,0.0,1.0,40.5,40.5,98.6,3,NO
272,Matt Ryan,26.0,35.0,74.3,374.0,10.7,5.0,0.0,3.0,4.0,12.0,0.0,0.0,1.0,40.2,40.2,94.3,3,ATL
447,Jared Goff,26.0,33.0,78.8,465.0,14.1,5.0,0.0,1.0,2.0,7.0,0.0,0.0,1.0,39.3,39.3,90.6,4,LAR
125,Ben Roethlisberger,39.0,60.0,65.0,452.0,7.5,3.0,0.0,1.0,2.0,9.0,1.0,0.0,1.0,39.0,39.0,95.1,2,PIT
217,Patrick Mahomes,23.0,28.0,82.1,326.0,11.6,6.0,0.0,1.0,5.0,18.0,0.0,0.0,1.0,38.8,38.8,98.7,2,KC
418,Andrew Luck,40.0,62.0,64.5,464.0,7.5,4.0,0.0,4.0,3.0,11.0,0.0,1.0,1.0,35.7,35.7,87.0,4,IND
177,Kirk Cousins,35.0,48.0,72.9,425.0,8.9,4.0,1.0,2.0,2.0,5.0,0.0,0.0,1.0,33.5,33.5,95.7,2,MIN
188,Blake Bortles,29.0,45.0,64.4,377.0,8.4,4.0,1.0,0.0,6.0,35.0,0.0,0.0,1.0,32.6,32.6,41.5,2,JAC


Through Week 5, no player has posted 2 of the season's Top 10 QB performances — yay parity!

In [118]:
qbdf.groupby('player')['fpoints'].mean().sort_values(ascending=False)[0:10]

player
Patrick Mahomes       26.22
Drew Brees            25.10
Matt Ryan             24.68
Jared Goff            22.96
Philip Rivers         22.36
Deshaun Watson        22.18
Kirk Cousins          21.94
Ben Roethlisberger    21.70
Aaron Rodgers         20.90
Andrew Luck           20.62
Name: fpoints, dtype: float64

Through Week 5, we see some familiar suspects in the Top 10 points per game, but also some young guns like Mahomes, Goff, Watson, Cousins, and a resurgence for Luck.

9. Now let's dump everything we have so far into a `csv` file that we can save to our current working directory:

In [136]:
qbdf.to_csv('./data/qb_stats_2018.csv')

___
### Running Backs
(Note that the tables are structured significantly differently for each position otherwise we would be able to use one nice and clean single scrape).

In [120]:
player = []

rush_att = []
rush_yds = []
rush_ypc = []
long = []
over_20 = []
rush_TD = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
rec_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [121]:
rb_stats_lists = [player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,
                  receptions,targets,rec_yds,rec_ypr,rec_TD,
                  fumbles_lost,active,fpoints,fpoints_g,own_pct]

for week_number in range(1,6):
    res = requests.get('https://www.fantasypros.com/nfl/stats/rb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(rb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

rbdf = pd.DataFrame(columns= ['player','rush_att','rush_yds','rush_ypc','long','over_20','rush_TD',
                  'receptions','targets','rec_yds','rec_ypr','rec_TD',
                  'fumbles_lost','active','fpoints','fpoints_g','own_pct'])
for index, column in enumerate(rbdf.columns):
    rbdf[column] = rb_stats_lists[index]

rbdf = table_cleaner(rbdf)

In [135]:
rbdf.to_csv('./data/rb_stats_2018.csv')

In [123]:
# Little spot check to make sure we're Gucci
rbdf.loc[rbdf['fpoints'] == rbdf.fpoints.max(), :]

,player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,receptions,targets,rec_yds,rec_ypr,rec_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
189,Alvin Kamara,8.0,29.0,3.6,10.0,0.0,2.0,9.0,12.0,112.0,12.4,1.0,0.0,1.0,38.6,38.6,100.0,1,NO
978,Alvin Kamara,19.0,134.0,7.1,49.0,1.0,3.0,5.0,9.0,47.0,9.4,0.0,0.0,1.0,38.6,38.6,100.0,4,NO


___
### Wide Receivers

In [124]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [125]:
wr_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,6):
    res = requests.get('https://www.fantasypros.com/nfl/stats/wr.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(wr_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

wrdf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(wrdf.columns):
    wrdf[column] = wr_stats_lists[index]

wrdf = table_cleaner(wrdf)

In [134]:
wrdf.to_csv('./data/wr_stats_2018.csv')

In [127]:
# Andddd our usual sanity check
wrdf.loc[wrdf['fpoints'] > 30, :]

,player,receptions,targets,rec_yds,rec_ypr,long,over_20,rec_TD,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
215,Tyreek Hill,7.0,8.0,169.0,24.1,58.0,5.0,2.0,2.0,4.0,0.0,0.0,1.0,32.8,32.8,99.9,1,KC
537,Stefon Diggs,9.0,13.0,128.0,14.2,75.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,31.4,31.4,99.8,2,MIN
1021,Calvin Ridley,7.0,8.0,146.0,20.9,75.0,1.0,3.0,1.0,9.0,0.0,0.0,1.0,37.0,37.0,90.1,3,ATL
1328,Cooper Kupp,9.0,11.0,162.0,18.0,70.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,32.7,32.7,94.9,4,LAR


___
### Tight Ends

In [128]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [129]:
te_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,6):
    res = requests.get('https://www.fantasypros.com/nfl/stats/te.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(te_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

tedf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(tedf.columns):
    tedf[column] = te_stats_lists[index]

tedf = table_cleaner(tedf)

In [132]:
tedf.to_csv('./data/te_stats_2018.csv')

In [131]:
# Chiggity check it
tedf.loc[tedf['fpoints'] > 25, :]

,player,receptions,targets,rec_yds,rec_ypr,long,over_20,rec_TD,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
285,Travis Kelce,7.0,10.0,109.0,15.6,31.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,26.4,26.4,100.0,2,KC
668,Jared Cook,8.0,13.0,110.0,13.8,24.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,27.0,27.0,92.5,4,OAK
944,Eric Ebron,9.0,15.0,105.0,11.7,28.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,27.0,27.0,84.9,5,IND


___
### Defense / Special Teams

In [139]:
player = []

sacks = []
INTs = []
fum_rec = []
force_fum = []
def_td = []
safeties = []
spc_TD = []

active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [140]:
dst_stats_lists = [player, sacks, INTs, fum_rec, force_fum, 
                   def_td, safeties, spc_TD, 
                   active, fpoints, fpoints_g, own_pct]

for week_number in range(1,6):
    res = requests.get('https://www.fantasypros.com/nfl/stats/dst.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(dst_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

dstdf = pd.DataFrame(columns= ['player', 'sacks', 'INTs', 'fum_rec', 'force_fum', 
                   'def_td', 'safeties', 'spc_TD', 
                   'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(dstdf.columns):
    dstdf[column] = dst_stats_lists[index]

dstdf = table_cleaner(dstdf)

In [141]:
dstdf.loc[dstdf.fpoints >= 20, :]

,player,sacks,INTs,fum_rec,force_fum,def_td,safeties,spc_TD,active,fpoints,fpoints_g,own_pct,week,team
21,New York Jets,0.0,5.0,0.0,0.0,1.0,0.0,1.0,1.0,20.0,20.0,0.0,1,NYJ
107,Green Bay Packers,7.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,23.0,23.0,0.0,4,GB
128,Arizona Cardinals,4.0,2.0,3.0,3.0,1.0,0.0,0.0,1.0,21.0,21.0,0.0,5,ARI
134,Cincinnati Bengals,3.0,2.0,1.0,1.0,2.0,0.0,0.0,1.0,22.0,22.0,0.0,5,CIN
143,Kansas City Chiefs,5.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,22.0,22.0,0.0,5,KC


Stats to bring in:
- ~~RBs~~
- ~~WRs~~
- ~~TEs~~
- DSTs
- Matchups (including game location)
- Defensive stats
- Team play-calling
- Depth charts?
- Weather